In [1]:
import os
import pandas as pd
import numpy as np
import math

from selenium.webdriver.common.alert import Alert
from selenium import webdriver  # 라이브러리(모듈) 가져오라
import chromedriver_autoinstaller

from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('data/data_제작비_한국.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df

,no.,title,directors,actors,plots,genre,runtime,keywords,등급,국적,screenCnt,audiAcc,salesAcc,개봉일,제작비
0,2.0,물괴,허종호,"김명민, 김인권, 혜리, 최우식, 이경영, 박희순, 박성웅, 이도경, 김중희, 이규...",“인왕산에 흉악한 짐승이 나타나 사람을 해쳤다 하옵니다.그것을 두고 듣지도 보지도 ...,액션,105.0,"조선시대,전염병,괴물,시대극/사극,사투",15세이상관람가,한국,1183.0,723414.0,6.104165e+09,2018-09-12,1.250000e+10
1,3.0,싱글라이더,이주영,"이병헌, 공효진, 안소희, 잭 캠벨, 양유진, 애니카 화이틀리, 케이 에크룬드, 백...",그가 사라졌다그에게서 모든 것이 사라졌다증권회사의 지점장 강재훈(이병헌). 안정된 ...,드라마,97.0,"기러기아빠,호주,오스트레일리아,반전,반전영화,동물",15세이상관람가,한국,576.0,351276.0,2.685519e+09,2017-02-22,4.500000e+09
2,7.0,허스토리,민규동,"김희애, 김해숙, 예수정, 문숙, 이용녀, 김선영, 김준한, 이유영, 이지하, 이용...","이겨야죠! 이겨야 할매들 분이 안풀리겠습니까? 1992~1998 6년의 기간, 2...",드라마,121.0,"종군위안부,위안부,일본군위안부피해자,실화바탕,재판,관부재판,정신대,정신대할머니,법정...",12세이상관람가,한국,696.0,336822.0,2.521026e+09,2018-06-27,2.500000e+09
3,9.0,풀잎들,홍상수,"김민희, 정진영, 기주봉, 서영화, 김새벽, 안재홍, 공민정, 안선영, 신석호, 김...",커피집이 있을 것 같지 않은 골목 안으로 커피집이 있고 사람들이 커피집 안 여기저기...,드라마,66.0,"삶과죽음,타인의 삶,언행,부산국제영화제,베를린영화제,삶의굴레,삶의순환,삶의의미",15세이상관람가,한국,54.0,7443.0,6.032025e+07,2018-10-25,1.000000e+08
4,13.0,영주,차성덕,"김향기, 김호정, 유재명, 탕준상, 장혜진, 김용준, 강동윤, 박옥출, 현봉식, 전...",나쁘고 좋은 사람들을 만났다.교통사고로 부모를 잃고 졸지에 가장이 된 영주는 자신의...,드라마,101.0,"부산국제영화제,부모님의죽음,장편데뷔작,복수극",12세이상관람가,한국,106.0,24823.0,1.927298e+08,2018-11-22,2.200000e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,6317.0,방황하는 칼날,이정호,"정재영, 이성민, 서준영, 김대명, 김현, 이수빈, 이주승, 최상욱, 김지혁, 김홍...","한 아이의 아버지는 하나뿐인 딸을 잃은 피해자가 되고, 이제는 살인자가 되었습니다....",범죄,122.0,"복수,살인자,소설원작,스릴러,아버지,윤간,일본소설원작,집단성폭행,추격,피해자,형사,...",청소년관람불가,한국,594.0,987420.0,7.806378e+09,2014-04-10,2.400000e+09
573,6320.0,히말라야,이석훈,"황정민, 정우, 조성하, 김인권, 라미란, 김원해, 이해영, 전배수, 정규수, 성병...","“기다려... 우리가 꼭 데리러 갈게...”해발 8,750 미터 히말라야 에베레스트...",드라마,124.0,"산,등산,히말라야,동료,에베레스트,실화바탕,실화바탕,실화바탕,실존인물,산악등반,동사...",12세이상관람가,한국,1009.0,7759473.0,6.017358e+10,2015-12-16,1.000000e+10
574,6334.0,허삼관,하정우,"하정우, 하지원, 남다름, 노강민, 전현석, 전혜진, 장광, 민무제, 주진모, 성동...","그나저나 나한테 언제 시집 오실 생각이세요? 돈 없고, 대책 없고, 가진 것도 ...",드라마,123.0,"소설원작,중국소설원작",12세이상관람가,한국,621.0,955175.0,7.405416e+09,2015-01-14,1.000000e+10
575,6338.0,해적: 바다로 간 산적,이석훈,"손예진, 유해진, 김원해, 박철민, 조달환, 신정근, 설리, 이이경, 이경영, 김경...",전국민을 시원하게 웃겨줄 액션 어드벤처!조선을 뒤흔든 최강도적들의 대격전! 조선의 ...,어드벤처,130.0,"고래,바다,사대부,조선,해적,국새",12세이상관람가,한국,838.0,8646758.0,6.624063e+10,2014-08-06,1.500000e+10


In [4]:
df_title = df['title']

In [5]:
df_title

0                물괴
1             싱글라이더
2              허스토리
3               풀잎들
4                영주
           ...     
572         방황하는 칼날
573            히말라야
574             허삼관
575    해적: 바다로 간 산적
576             숫호구
Name: title, Length: 577, dtype: object

In [6]:
chrome_path = chromedriver_autoinstaller.install()

driver = webdriver.Chrome(chrome_path)

driver.get('https://movie.naver.com/')

time.sleep(2)

In [8]:
name = driver.find_element_by_css_selector('.ipt_tx_srch')
name.clear()
name.send_keys('강남')

In [9]:
movie = driver.find_element_by_css_selector('.thumb_spec')
movie.click()

In [10]:
movie_score = driver.find_element_by_css_selector('.exp_cnt').get_attribute('textContent')

In [11]:
movie_score

'보고싶어요7'

In [22]:
extract = re.compile('[0-9]*,*[0-9]+')
str_num = extract.findall(movie_score)
str_num = str_num[0]
if ',' in str_num:
    str_num = str_num.split(',')
    str_num = int(str_num[0] + str_num[1])
    num = int(str_num)
else:
    num = int(str_num)
num

7

In [35]:
def naver_expectation(m_title):
    
    chrome_path = chromedriver_autoinstaller.install()
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    options.add_argument('windows-size=1920x1080')
    driver = webdriver.Chrome(chrome_path, options=options)
    
    driver.get('https://movie.naver.com/')
    time.sleep(1)


    name = driver.find_element_by_css_selector('.ipt_tx_srch')
    name.clear()
    name.send_keys(m_title)
    time.sleep(1)

    # 영화 클릭 
    movie = driver.find_element_by_css_selector('.thumb_spec')
    movie.click()
    time.sleep(2)

    # 기대지수 
    movie_score = driver.find_element_by_css_selector('.exp_cnt').get_attribute('textContent')
    time.sleep(1)

    extract = re.compile('[0-9]*,*[0-9]+')
    str_num = extract.findall(movie_score)
    str_num = str_num[0]
    if ',' in str_num:
        str_num = str_num.split(',')
        str_num = int(str_num[0] + str_num[1])
        num = int(str_num)
    else:
        num = int(str_num)
    
    return num

In [38]:
naver_expectation = naver_expectation('강남')

In [39]:
naver_expectation

7

In [ ]:
dict = {}
chrome_path = chromedriver_autoinstaller.install()

webdriver_options = webdriver.ChromeOptions()
webdriver_options .add_argument('headless')

driver = webdriver.Chrome(chrome_path, options=webdriver_options)


driver.get('https://movie.naver.com/')
time.sleep(2)

for i in tqdm_notebook(range(len(df_title))):
    # 영화 검색창 클릭 및 입력
    try:
        name = driver.find_element_by_css_selector('.ipt_tx_srch')
        time.sleep(0.5)
        name.click()
        time.sleep(0.5)
        name.clear()
        time.sleep(0.5)
        name.send_keys(df_title[i])
        time.sleep(1)

        # 영화 클릭 
        movie = driver.find_element_by_css_selector('.thumb_spec')
        movie.click()
        time.sleep(2)
        
        # 영화 제목
        title = df_title[i]
        time.sleep(1)
        
        # 기대지수 
        movie_score = driver.find_element_by_css_selector('.exp_cnt').get_attribute('textContent')
        time.sleep(1)
        
        extract = re.compile('[0-9]*,*[0-9]+')
        str_num = extract.findall(movie_score)
        str_num = str_num[0]
        if ',' in str_num:
            str_num = str_num.split(',')
            str_num = int(str_num[0] + str_num[1])
            num = int(str_num)
        else:
            num = int(str_num)
        num
        
        time.sleep(1)
        
        total = {}

        total['title'] = title
        total['movie_score'] = num
        time.sleep(1)
        
        dict[i] = total
        time.sleep(1)
        
        print(title, num)
        
    except:
        pass
    

In [7]:
data = pd.DataFrame.from_dict(dict, 'index')

In [8]:
data

,title,movie_score
0,물괴,1274
1,싱글라이더,3838
2,허스토리,494
3,풀잎들,162
4,영주,800
...,...,...
572,방황하는 칼날,1932
573,히말라야,13693
574,허삼관,10731
575,해적: 바다로 간 산적,4941


In [9]:
data.to_csv('movie_buzz/naver_movie_score.csv', encoding='utf-8-sig')

In [18]:
data = pd.read_csv('movie_buzz/naver_movie_기대지수.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data

,title,movie_score
0,물괴,1274
1,싱글라이더,3838
2,허스토리,494
3,풀잎들,162
4,영주,800
...,...,...
570,방황하는 칼날,1932
571,히말라야,13693
572,허삼관,10731
573,해적: 바다로 간 산적,4941


In [15]:
# 잘못들어간 데이터 변경
data.loc[data['title'] == '목격자', 'movie_score'] = 583
data.loc[data['title'] == '형', 'movie_score'] = 25600
data.loc[data['title'] == '인랑', 'movie_score'] = 2941
data.loc[data['title'] == '범죄도시', 'movie_score'] = 7567
data.loc[data['title'] == '데자뷰', 'movie_score'] = 159
data.loc[data['title'] == '챔피언', 'movie_score'] = 726
data.loc[data['title'] == '변신', 'movie_score'] = 336
data.loc[data['title'] == '프리즌', 'movie_score'] = 8022
data.loc[data['title'] == '공작', 'movie_score'] = 2437
data.loc[data['title'] == '사냥', 'movie_score'] = 5209
data.loc[data['title'] == '럭키', 'movie_score'] = 15767
data.loc[data['title'] == '우리들', 'movie_score'] = 1035
data.loc[data['title'] == '아수라', 'movie_score'] = 28898
data.loc[data['title'] == '리얼', 'movie_score'] = 25803
data.loc[data['title'] == '인천상륙작전', 'movie_score'] = 35317
data.loc[data['title'] == '소리꾼', 'movie_score'] = 0
data.loc[data['title'] == '우리집', 'movie_score'] = 149
data.loc[data['title'] == '부라더', 'movie_score'] = 3484
data.loc[data['title'] == '더 킹', 'movie_score'] = 17679
data.loc[data['title'] == '침묵', 'movie_score'] = 5381
data.loc[data['title'] == '강철비', 'movie_score'] = 5965
data.loc[data['title'] == '사자', 'movie_score'] = 754
data.loc[data['title'] == '아이', 'movie_score'] = 1218
data.loc[data['title'] == '돈', 'movie_score'] = 2278
data.loc[data['title'] == '베를린', 'movie_score'] = 0
data.loc[data['title'] == '화차', 'movie_score'] = 0
data.loc[data['title'] == '무서운 이야기', 'movie_score'] = 0
data.loc[data['title'] == '더 테러 라이브', 'movie_score'] = 0
data.loc[data['title'] == '내부자들', 'movie_score'] = 12198
data.loc[data['title'] == '순정', 'movie_score'] = 31698
data.loc[data['title'] == '귀향', 'movie_score'] = 47687
data.loc[data['title'] == '대호', 'movie_score'] = 8397
data.loc[data['title'] == '검은사제들', 'movie_score'] = 15918
data.loc[data['title'] == '오빠생각', 'movie_score'] = 8257
data.loc[data['title'] == '사도', 'movie_score'] = 22459
data.loc[data['title'] == '화장', 'movie_score'] = 3583
data.loc[data['title'] == '용의자', 'movie_score'] = 0
data.loc[data['title'] == '소녀괴담', 'movie_score'] = 3750
data.loc[data['title'] == '신의 한 수', 'movie_score'] = 2407

- 국적이 잘못되어있는 영화
    - 숨바꼭질 index: 37
    - 사이코메트리 index: 159

In [20]:
data.loc[data['title'] == '스파이']

,title,movie_score
457,스파이,45780


In [22]:
data.loc[data['title'] == '대호']

,title,movie_score
477,대호,8397


In [24]:
data.loc[37]

title          숨바꼭질
movie_score      24
Name: 37, dtype: object

In [25]:
data.loc[159]

title          사이코메트리
movie_score        46
Name: 159, dtype: object

In [15]:
data.loc[457]

Unnamed: 0       457
title            스파이
movie_score    45780
Name: 457, dtype: object

In [26]:
data.drop([37, 159], inplace=True)

In [21]:
data.drop(457, inplace=True)

In [29]:
data.loc[36:38]

,title,movie_score
36,봉이 김선달,20055
38,영하의 바람,71


In [30]:
data.loc[158:160]

,title,movie_score
158,나홀로 휴가,946
160,조작된 도시,12267


In [22]:
data.loc[456:458]

,title,movie_score
456,경성학교: 사라진 소녀들,8640
458,내부자들,12198


In [23]:
data.reset_index(drop=True, inplace=True)
data

,title,movie_score
0,물괴,1274
1,싱글라이더,3838
2,허스토리,494
3,풀잎들,162
4,영주,800
...,...,...
569,방황하는 칼날,1932
570,히말라야,13693
571,허삼관,10731
572,해적: 바다로 간 산적,4941


In [24]:
data.to_csv('movie_buzz/naver_movie_기대지수.csv', encoding='utf-8-sig')